# Compute the bbox polygons for ICESat-2 transects
This notebook reads in a saved geopackage storing sICESat-2 ATL08 data and outputs a geopackage storing polygons of size 100 x 12 meters along the ICESat-2 scan line, surrounding the data point.

In [1]:
import geopandas as gpd
import math
from shapely.geometry import MultiLineString, LineString, Point, Polygon
import ee
import geemap as gm

In [2]:
#ee.Authenticate()

In [3]:
#ee.Initialize()

## Import ICESat2 saved in geopackage
This geopackage only stores the lat and long of the center of each transect. 

Also needed track id (eg gt3l) - the track of the previous and next point needs to be the same, otherwise don't create a polygon.

The rest of the attributes are stored separately.

In [2]:
# import ICESat-2 transects that are saved in a GeoPackage
gdf_est = gpd.read_file("..\\Data\\icesat_data\\icesat2_est.gpkg", layer='all_icesat2_data')  
gdf_est.head()

,track,seg,timestamp,year,month,beam_nr,beam_t,can_h,n_can_pho,n_topcan_pho,...,dem_h,night,solar_el,snow,brightness,cloud,urban,lon,lat,geometry
0,gt1l,670809,2.471818e+07,2018,10,6,weak,9.191999,24,6,...,23.925793,1,-21.467110,0,0,3,0,550531.996546,6.600682e+06,POINT (550531.997 6600681.958)
1,gt1l,670864,2.471818e+07,2018,10,6,weak,13.351807,22,4,...,28.396036,1,-21.471407,0,0,3,0,550433.908225,6.599585e+06,POINT (550433.908 6599584.921)
2,gt1l,670869,2.471818e+07,2018,10,6,weak,14.949738,27,8,...,31.422529,1,-21.471790,1,0,3,1,550425.083941,6.599485e+06,POINT (550425.084 6599485.347)
3,gt1l,670914,2.471818e+07,2018,10,6,weak,21.750122,38,13,...,52.339931,1,-21.475302,0,0,3,0,550344.254889,6.598588e+06,POINT (550344.255 6598587.886)
4,gt1l,670934,2.471818e+07,2018,10,6,weak,20.947483,36,5,...,45.463993,1,-21.476864,0,0,3,0,550308.290920,6.598189e+06,POINT (550308.291 6598189.157)


## Get polygon bbox of ICESat-2 point 

Used example from https://glenbambrick.com/tag/shapely/

In [8]:
def getAngle(pt1, pt2):
    x_diff = pt2.x - pt1.x
    y_diff = pt2.y - pt1.y
    return math.degrees(math.atan2(y_diff, x_diff))

## start and end points of chainage tick
## get the first end point of a tick
def getPoint1(pt, bearing, dist):
    angle = bearing + 90
    bearing = math.radians(angle)
    x = pt.x + dist * math.cos(bearing)
    y = pt.y + dist * math.sin(bearing)
    return Point(x, y)


## get the second end point of a tick
def getPoint2(pt, bearing, dist):
    bearing = math.radians(bearing)
    x = pt.x + dist * math.cos(bearing)
    y = pt.y + dist * math.sin(bearing)
    return Point(x, y)


For each ICESat-2 point:
1. Create one line from the ICESat-2 point to the previous point and another line to the next point
2. For both, find an edge point that lies 50 meters from the current point along each of the line created -> bbox 100 m across the scan line
3. For both of the new edge points find two points that lie at 90 degrees from the scan line on each side -> corners of the bbox
4. Save the four corner points as Polygon to the dataframe

In [9]:
# can test with the small dataframe (gdfs) - assign dataframe here
dataframe = gdf_est
for index, row in dataframe.iterrows():
    
    # first ignore the first and last point in the dataframe
    if index == 0 or index == len(dataframe)-1:
        continue
    
    # ignore if the previous or the next data point in dataframe belongs to a different ground track
    # here assumption is made that the same track from different scans are not consecutive in dataframe
    if dataframe['track'][index] != dataframe['track'][index-1] or dataframe['track'][index] != dataframe['track'][index+1]:
        continue
    else:
        
        # create a line to the previous point and to the next icesat point
        line_before = LineString([dataframe['geometry'][index], dataframe['geometry'][index-1]])
        line_next = LineString([dataframe['geometry'][index], dataframe['geometry'][index+1]])
        
        # find a point to mid-way to the previous point and to the next point
        interpolation_bef = line_before.interpolate(50)
        interpolation_next = line_next.interpolate(50)
        
        # first find the two points 90 degrees from the point half way to the previous point
        angle = getAngle(interpolation_bef, dataframe['geometry'][index-1])
        # create point at 90 degrees angle 6 meters from the transect line
        end_1 = getPoint1(interpolation_bef, angle, 6)
        # and to the other side
        angle = getAngle(end_1, interpolation_bef)
        end_2 = getPoint2(interpolation_bef, angle, 6)
        
        # now two points towards the next point
        angle = getAngle(interpolation_next, dataframe['geometry'][index+1])
        end_3 = getPoint1(interpolation_next, angle, 6)
        angle = getAngle(end_3, interpolation_next)
        end_4 = getPoint2(interpolation_next, angle, 6)
        
        # now store this as Polygon
        poly = Polygon([(end_1.x, end_1.y), (end_2.x, end_2.y), (end_3.x, end_3.y), (end_4.x, end_4.y), (end_1.x, end_1.y)])

        dataframe.loc[index, 'polygon']=poly
        #print(poly)
        
        

In [10]:
#gdfs
gdf_est.head()

,track,seg,timestamp,year,month,beam_nr,beam_t,can_h,n_can_pho,n_topcan_pho,...,night,solar_el,snow,brightness,cloud,urban,lon,lat,geometry,polygon
0,gt1l,670809,2.471818e+07,2018,10,6,weak,9.191999,24,6,...,1,-21.467110,0,0,3,0,550531.996546,6.600682e+06,POINT (550531.997 6600681.958),NaN
1,gt1l,670864,2.471818e+07,2018,10,6,weak,13.351807,22,4,...,1,-21.471407,0,0,3,0,550433.908225,6.599585e+06,POINT (550433.908 6599584.921),"POLYGON ((550432.3849052369 6599635.256918412,..."
2,gt1l,670869,2.471818e+07,2018,10,6,weak,14.949738,27,8,...,1,-21.471790,1,0,3,1,550425.083941,6.599485e+06,POINT (550425.084 6599485.347),POLYGON ((550423.5210649766 6599535.6812770255...
3,gt1l,670914,2.471818e+07,2018,10,6,weak,21.750122,38,13,...,1,-21.475302,0,0,3,0,550344.254889,6.598588e+06,POINT (550344.255 6598587.886),"POLYGON ((550342.7641310489 6598638.222706819,..."
4,gt1l,670934,2.471818e+07,2018,10,6,weak,20.947483,36,5,...,1,-21.476864,0,0,3,0,550308.290920,6.598189e+06,POINT (550308.291 6598189.157),"POLYGON ((550306.8067753656 6598239.494293771,..."


## Export the ATL08 transect bbox-s as polygons 

In [12]:
# create a dataframe with only the polygons
polygons_all = gpd.GeoDataFrame(data = gdf_est.drop(columns=['geometry']), geometry =gdf_est['polygon'], crs= 'EPSG:3301')
polygons = polygons_all[polygons_all['geometry'].notnull()] 

polygons = polygons.drop(columns=['polygon'])

polygons[:5]

,track,seg,timestamp,year,month,beam_nr,beam_t,can_h,n_can_pho,n_topcan_pho,...,dem_h,night,solar_el,snow,brightness,cloud,urban,lon,lat,geometry
1,gt1l,670864,2.471818e+07,2018,10,6,weak,13.351807,22,4,...,28.396036,1,-21.471407,0,0,3,0,550433.908225,6.599585e+06,"POLYGON ((550432.385 6599635.257, 550444.337 6..."
2,gt1l,670869,2.471818e+07,2018,10,6,weak,14.949738,27,8,...,31.422529,1,-21.471790,1,0,3,1,550425.083941,6.599485e+06,"POLYGON ((550423.521 6599535.681, 550435.474 6..."
3,gt1l,670914,2.471818e+07,2018,10,6,weak,21.750122,38,13,...,52.339931,1,-21.475302,0,0,3,0,550344.254889,6.598588e+06,"POLYGON ((550342.764 6598638.223, 550354.716 6..."
4,gt1l,670934,2.471818e+07,2018,10,6,weak,20.947483,36,5,...,45.463993,1,-21.476864,0,0,3,0,550308.290920,6.598189e+06,"POLYGON ((550306.807 6598239.494, 550318.758 6..."
5,gt1l,670939,2.471818e+07,2018,10,6,weak,22.216278,34,8,...,42.452492,1,-21.477251,0,0,3,0,550299.464753,6.598089e+06,"POLYGON ((550297.884 6598139.493, 550309.838 6..."


In [15]:
# export to geopackage
polygons.to_file('..\\Data\\ICESat2_polygons\\atl08_polygons.gpkg', driver='GPKG', layer='ALS_08_all_polygons')

C:\Users\Kati\anaconda3\envs\thesis\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## The following analysis was completed in QGIS
The following steps were takin in QGIS:
1. The exported ICESat-2 polygons were added to QGIS together with a forest cover dataset by Hansen et al. (2013) and the dominant tree species dataset by Lang et al. (2019).
2. The geometries of the polygons was fixed using 'fix geometries' tool
3. Only the ICESat-2 polygons which contained forest that according to Hansen et al. (2013) has not had any clear cuts since the year 2000 and which has at least 30% of forest cover were selected -> forested ICESat-2 polygons. This was done using sum  statistics in Zonal Statistics tool finding the sum of the pixels where intact forest = 1. 
3. Then, only the polygons that had the sum greater than 0 were saved and the dominant tree species was found. Using the Zonal Statistics tool for only the polygons that contained forest, a dominant tree type was found in each polygon through finding the majority statistics showing the most represented pixel value. In addition, the variety - count of the number of distinct pixel values - was found for each ICESat-2 polygon, showing the number of different dominant tree species within the polygon. This layer was saved to geopackage as 'atl08_pol_species'.
4. A vector layer from Estonian Land Board was added that shows the 1:2000 map tiles with their respective IDs. The tiles that intersect with forested ICESat-2 polygons were selected using extract by location and intersection option and exported as separate geopackage. For each map square, there is a point cloud in laz format that is downloadable from Estonian Land Board Geoportal. This geopackage was used got downloading the necessary files. Saved as CSV. Not needed for the next step though.
5. A join between the ICESat-2 transects and the mapsquares was made so that for each transect, it was saved which map square (NR) it intersected with. Join by location tool was used. Important to note, that transects that intersected with more than one map square were saved multiple times (one-to-many). Saved as 'ATL08_with_ALS_duplicates' layer in geopackage.

## Show the forested ICESat-2 polygons on a map
This map is to show the data exported from QGIS. Zoom in to see the ATL08 polygons.

In [6]:
Map = gm.Map(zoom=25)
Map.setOptions('SATELLITE')

# prepare the layer showing forest with no clear cuts, by leaving only areas that have treecover 30% and have had no loss since 2000
gfc = ee.Image("UMD/hansen/global_forest_change_2021_v1_9")

intact_forest = gfc.expression("(b('loss') == 0) * (b('treecover2000') > 30)").selfMask().rename('intact_forest')
dominanttree=  ee.Image('projects/ee-oliverlevers/assets/Estonia_ForestSpecies').updateMask(intact_forest)

# add the ATL08 polygons
forested_polygons = ee.FeatureCollection('projects/ee-msc-thesis/assets/ATL08_polygons_forest_species')

Map.addLayer(intact_forest, {'palette':'green'}, 'Forest without clear cuts (Hansen 2013)', False)
Map.addLayer(dominanttree, {'min': 10,'max': 25, 'palette': ['grey', 'green', 'yellow', 'orange', 'red']}, 'Dominant tree species')

# to visualize the polygons better
empty = ee.Image().byte()
Map.addLayer(empty.paint(forested_polygons, '_majority'), {'palette': ['yellow'], max: 17}, 'Forested polygons');
Map.centerObject(forested_polygons)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…